In [1]:
import numpy as np
import torch

### 叔叔我啊，要开始训练了捏

首先，获取模型和数据


In [2]:
from dataset import MyDataset
from model import MyTransformerEncoder

model = MyTransformerEncoder(d_model=100, output_dimension=1)
train_data = np.load("./npys/train/sentence_code.npy")
train_labels = np.load("./npys/train/labels.npy")
vocabulary_vectors = np.load("./npys/vocabulary_vectors.npy")
dataset = MyDataset(train_data, train_labels, vocabulary_vectors)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, num_workers=8)

In [4]:
for X, y in iter(dataloader):
    print(X, X.shape, y)
    break

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 